In [2]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re
# 读取文件
f = open('./weibos.txt', 'r', encoding='UTF-8')
text = f.read()
# 以句号，叹号，问号作为分隔，去掉\n换行符号
sentences = re.split('[。！？]', text.replace('\n', ''))
# 最后一行如果为空，则删除
if sentences[len(sentences)-1] == '':
    sentences.pop()

In [5]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

In [6]:
# 设置停用词
stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/1c/n1hzp__5263fbm3xdyzk8xgw0000gn/T/jieba.cache
Loading model cost 0.807 seconds.
Prefix dict has been built successfully.


# 想用tfidf来替代N-gram，想到的方法是将每一句做tfidf得到的feature传入MinHash中，不知道想法是否正确，效果好像还行

In [51]:
vec = TfidfVectorizer()
# 对item_str创建MinHash
def get_minhash(item_str):
    print(item_str)
    D = vec.fit_transform(item_str)
    voc = dict((i, w) for w, i in vec.vocabulary_.items())
    temp = MinHash()
    for i in range(D.shape[0]):
        Di = D.getrow(i)
        features = [voc[j] for j in Di.indices]
        print(list(features))
        features_list = list(features)
        for i in range(len(features_list)):
            temp.update(str(features_list[i]).encode('utf8'))
    return temp
get_minhash([documents[16]])


['斯科拉里 ， 看好 的 不是 国 足 ， 而是 年薪 … …   \u200b 非常 应该 辞职 ']
['辞职', '应该', '非常', '年薪', '而是', '不是', '看好', '斯科拉里']


In [44]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash([documents[i]])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

['# 斯科拉里 愿意 执教 国足 # 上 一届 如果 是 里 皮 从头 芾 到 尾 ， 是 很大 机会 入 世界杯 的 ， 这 一届 ， 没 几个 能用 的 ， 除非 大力 归化 ， 谁 来 都 没用 ']
  (0, 13)	0.22941573387056174
  (0, 7)	0.22941573387056174
  (0, 5)	0.22941573387056174
  (0, 15)	0.22941573387056174
  (0, 14)	0.22941573387056174
  (0, 3)	0.22941573387056174
  (0, 1)	0.22941573387056174
  (0, 12)	0.22941573387056174
  (0, 8)	0.22941573387056174
  (0, 2)	0.22941573387056174
  (0, 6)	0.22941573387056174
  (0, 0)	0.4588314677411235
  (0, 4)	0.22941573387056174
  (0, 10)	0.22941573387056174
  (0, 9)	0.22941573387056174
  (0, 11)	0.22941573387056174
{11: '斯科拉里', 9: '愿意', 10: '执教', 4: '国足', 0: '一届', 6: '如果', 2: '从头', 8: '很大', 12: '机会', 1: '世界杯', 3: '几个', 14: '能用', 15: '除非', 5: '大力', 7: '归化', 13: '没用'}
huangw[0.22941573 0.22941573 0.22941573 0.22941573 0.22941573 0.22941573
 0.22941573 0.22941573 0.22941573 0.22941573 0.22941573 0.45883147
 0.22941573 0.22941573 0.22941573 0.22941573]
['没用', '归化', '大力', '除非', '能用', '几个', '世界杯', '机会', '很大', '从头', '如果', '一届', '国足', '执教', 

In [52]:
#query = '不看好国足'
#query = '一个比一个水，国足踢不好完全是足协的问题，足协不解散重组'
query='里皮能带走出迷茫'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash([item_str])

# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

['里 皮 能带 走出 迷茫 ']
['迷茫', '走出', '能带']
29 0.0625 ​带队结果绝不会比里皮更差，​而且我一定能带国足夺得2022世界杯冠军
31 0.140625 ​曾经也以为世界名帅里皮能够带领国足走出迷茫，让国足蒸蒸日上，也让我们国人不再迷茫，吃下一颗定心丸
Top 3 邻居 [29, 31]
